In [16]:
import numpy as np
import os
import pandas as pd
import pandas as pd
from pandas import json_normalize
import ast
import json
import re
import yaml

In [17]:
t7_drive = '/media/ak/T7/August11th2022Experiments/'
infoClockFolders = 'InfoClockDataFrames'
mmdOutputFiles = 'LinearMMDOutputFiles'


In [3]:
def parse_string_to_dict(s):
    s = re.sub(r"(\w+):", r'"\1":', s)  # Add quotes around keys
    s = s.replace("'", '"')  # Replace single quotes with double quotes
    return json.loads(s)  # Parse the resulting JSON string

In [18]:
mmdOutputFilesLoc = os.path.join(t7_drive, mmdOutputFiles)

In [19]:
csvfiles = os.listdir(mmdOutputFilesLoc)

In [20]:
csvfile = os.path.join(mmdOutputFilesLoc, 'G_1_volume_processedLinearMMDresults_alpha.pickle')

In [7]:
df = pd.read_csv(csvfile)

In [8]:
df = df.set_index(['Column Pair', 'Window', 'Shift'])

In [9]:
def fix_string_issues(s):
    s = s.replace("array(", "[")
    s = s.replace(", dtype=float64)", "]")
    s = s.replace(", 'besti'", "], 'besti'")
    s = re.sub(r"(\w+):", r'"\1":', s)  # Add quotes around keys
    s = s.replace("'", '"')  # Replace single quotes with double quotes
    s = s.replace("],]", "]]")  # Fix the remaining issue
    return s

In [12]:
def fix_string_issues(s):
    s = s.replace("array(", "[")
    s = s.replace(", dtype=float64)", "]")
    s = re.sub(r"(\w+):", r'"\1":', s)  # Add quotes around keys
    s = s.replace("'", '"')  # Replace single quotes with double quotes
    return s

def fix_and_parse_json(s):
    s = re.sub(r'\s+', ' ', s)  # Replace newline characters and extra spaces with single spaces
    s = re.sub(r'] ({)', r'], \1', s)  # Add missing comma between key-value pairs
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        print(f"Error parsing: {s}")
        raise e



In [21]:
# df['Test Results 1'] = df['Test Results 1'].apply(fix_and_parse_json)
# df['Test Results 2'] = df['Test Results 2'].apply(fix_and_parse_json)
csvfile


'/media/ak/T7/August11th2022Experiments/LinearMMDOutputFiles/G_1_volume_processedLinearMMDresults_alpha.pickle'

In [22]:
import pickle
import pandas as pd
import numpy as np

# Load the pickle file
with open(str(csvfile), "rb") as f:
    result_list = pickle.load(f)

# Convert the result_list to a DataFrame
df = pd.DataFrame(result_list)

In [23]:
df

,Column Pair,Window,Shift,Test Results 1,Test Results 2
0,0 vs 2,window=5,shift=1,"{'widths': [0.3127758084013007, 0.351078974559...","{'test_result': {'alpha': 0.01, 'pvalue': 6.81..."
1,0 vs 2,window=10,shift=1,"{'widths': [0.3127758084013007, 0.351078974559...","{'test_result': {'alpha': 0.01, 'pvalue': nan,..."
2,0 vs 2,window=15,shift=1,"{'widths': [0.3127758084013007, 0.351078974559...","{'test_result': {'alpha': 0.01, 'pvalue': 9.05..."
3,0 vs 2,window=20,shift=1,"{'widths': [0.3127758084013007, 0.351078974559...","{'test_result': {'alpha': 0.01, 'pvalue': 6.06..."
4,0 vs 2,window=25,shift=1,"{'widths': [0.3127758084013007, 0.351078974559...","{'test_result': {'alpha': 0.01, 'pvalue': 1.24..."
...,...,...,...,...,...
940,31 vs 37,window=5,shift=5,"{'widths': [0.2611623442839497, 0.293144819906...","{'test_result': {'alpha': 0.01, 'pvalue': 3.19..."
941,31 vs 35,window=5,shift=2,"{'widths': [0.4266564188107409, 0.478905637782...","{'test_result': {'alpha': 0.01, 'pvalue': 2.11..."
942,31 vs 36,window=5,shift=3,"{'widths': [0.43012312048811174, 0.48279687884...","{'test_result': {'alpha': 0.01, 'pvalue': nan,..."
943,32 vs 34,window=5,shift=1,"{'widths': [0.2900861974585337, 0.325610747385...","{'test_result': {'alpha': 0.01, 'pvalue': 2.23..."


In [24]:


# Extract relevant data from the nested dictionaries
df['alpha_1'] = df['Test Results 1'].apply(lambda x: x['alpha'])
df['pvalue_1'] = df['Test Results 1'].apply(lambda x: x['pvalue'])
df['test_stat_1'] = df['Test Results 1'].apply(lambda x: x['test_stat'])
df['h0_rejected_1'] = df['Test Results 1'].apply(lambda x: x['h0_rejected'])

df['alpha_2'] = df['Test Results 2'].apply(lambda x: x['alpha'])
df['pvalue_2'] = df['Test Results 2'].apply(lambda x: x['pvalue'])
df['test_stat_2'] = df['Test Results 2'].apply(lambda x: x['test_stat'])
df['h0_rejected_2'] = df['Test Results 2'].apply(lambda x: x['h0_rejected'])

# Perform calculations on the DataFrame
df['avg_alpha'] = (df['alpha_1'] + df['alpha_2']) / 2
df['avg_pvalue'] = (df['pvalue_1'] + df['pvalue_2']) / 2
df['avg_test_stat'] = (df['test_stat_1'] + df['test_stat_2']) / 2
df['h0_rejected_both'] = df['h0_rejected_1'] & df['h0_rejected_2']

# Compute mean and standard deviation for alpha, pvalue, and test_stat
mean_alpha = np.mean(df['avg_alpha'])
std_alpha = np.std(df['avg_alpha'])

mean_pvalue = np.mean(df['avg_pvalue'])
std_pvalue = np.std(df['avg_pvalue'])

mean_test_stat = np.mean(df['avg_test_stat'])
std_test_stat = np.std(df['avg_test_stat'])

print("Mean alpha:", mean_alpha, "Std alpha:", std_alpha)
print("Mean pvalue:", mean_pvalue, "Std pvalue:", std_pvalue)
print("Mean test_stat:", mean_test_stat, "Std test_stat:", std_test_stat)

# Check the results
print(df.head())


KeyError: 'alpha'

In [25]:
print(df['Test Results 1'][0])

{'widths': [0.3127758084013007, 0.35107897455974396, 0.39407282490268447, 0.44233179022332814, 0.49650064728642873, 0.5573031335400543, 0.6255516168026014, 0.7021579491194878, 0.7881456498053689, 0.8846635804466563, 0.9930012945728575, 1.1146062670801087, 1.2511032336052028, 1.4043158982389756, 1.5762912996107377, 1.7693271608933125, 1.986002589145715, 2.229212534160217, 2.5022064672104056, 2.8086317964779512, 3.1525825992214753, 3.538654321786625, 3.97200517829143, 4.458425068320434, 5.004412934420811], 'med': 0.3127758084013007, 'besti': 24, 'powers': array([0.92303857, 0.92347337, 0.92381611, 0.92408662, 0.92430036,
       0.92446937, 0.92460312, 0.92470902, 0.92479292, 0.92485941,
       0.92491211, 0.9249539 , 0.92498705, 0.92501334, 0.9250342 ,
       0.92505074, 0.92506387, 0.92507429, 0.92508256, 0.92508912,
       0.92509433, 0.92509846, 0.92510174, 0.92510434, 0.92510641]), 'med_on_test_data': 0.2761801866119382, 'test_result': {'alpha': 0.01, 'pvalue': 3.77273689349796e-06, 

In [26]:
# Extract relevant data from the nested dictionaries
df['alpha_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['alpha'])
df['pvalue_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['pvalue'])
df['test_stat_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['test_stat'])
df['h0_rejected_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['h0_rejected'])

df['alpha_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['alpha'])
df['pvalue_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['pvalue'])
df['test_stat_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['test_stat'])
df['h0_rejected_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['h0_rejected'])

# Drop unnecessary columns
df = df.drop(columns=['Test Results 1', 'Test Results 2'])

# Print the DataFrame
print(df.head())


  Column Pair     Window    Shift  alpha_1  pvalue_1  test_stat_1  \
0      0 vs 2   window=5  shift=1     0.01  0.000004     0.000278   
1      0 vs 2  window=10  shift=1     0.01       NaN          NaN   
2      0 vs 2  window=15  shift=1     0.01  0.000070     0.001134   
3      0 vs 2  window=20  shift=1     0.01  0.001449     0.000178   
4      0 vs 2  window=25  shift=1     0.01  0.000009     0.000902   

   h0_rejected_1  alpha_2  pvalue_2  test_stat_2  h0_rejected_2  
0           True     0.01  0.000007     0.000517           True  
1          False     0.01       NaN          NaN          False  
2           True     0.01  0.000009     0.000733           True  
3           True     0.01  0.000006     0.000359           True  
4           True     0.01  0.000001     0.000624           True  


In [27]:
df

,Column Pair,Window,Shift,alpha_1,pvalue_1,test_stat_1,h0_rejected_1,alpha_2,pvalue_2,test_stat_2,h0_rejected_2
0,0 vs 2,window=5,shift=1,0.01,0.000004,0.000278,True,0.01,0.000007,0.000517,True
1,0 vs 2,window=10,shift=1,0.01,NaN,NaN,False,0.01,NaN,NaN,False
2,0 vs 2,window=15,shift=1,0.01,0.000070,0.001134,True,0.01,0.000009,0.000733,True
3,0 vs 2,window=20,shift=1,0.01,0.001449,0.000178,True,0.01,0.000006,0.000359,True
4,0 vs 2,window=25,shift=1,0.01,0.000009,0.000902,True,0.01,0.000001,0.000624,True
...,...,...,...,...,...,...,...,...,...,...,...
940,31 vs 37,window=5,shift=5,0.01,0.000004,0.000639,True,0.01,0.000003,0.000623,True
941,31 vs 35,window=5,shift=2,0.01,0.000004,0.099437,True,0.01,0.000002,0.112520,True
942,31 vs 36,window=5,shift=3,0.01,0.000004,0.097917,True,0.01,NaN,0.000000,False
943,32 vs 34,window=5,shift=1,0.01,0.000006,0.088384,True,0.01,0.000002,0.310410,True


In [28]:
# Group by 'Column Pair' and 'Window' and calculate the mean p-value and proportion of H0 rejected
summary_table = df.groupby(['Column Pair', 'Window']).agg({
    'pvalue_1': 'mean',
    'h0_rejected_1': 'mean',
    'pvalue_2': 'mean',
    'h0_rejected_2': 'mean'
}).reset_index()

# Format column names
summary_table.columns = [
    'Column Pair', 'Window',
    'Mean P-Value 1', 'Proportion H0 Rejected 1',
    'Mean P-Value 2', 'Proportion H0 Rejected 2'
]

# Round the values for better readability
summary_table = summary_table.round(4)

# Display the summary table
print(summary_table)



    Column Pair     Window  Mean P-Value 1  Proportion H0 Rejected 1  \
0       0 vs 10  window=10             NaN                     False   
1       0 vs 10  window=15             NaN                     False   
2       0 vs 10  window=20             NaN                     False   
3       0 vs 10  window=25          0.0001                      True   
4       0 vs 10  window=30          0.0000                      True   
..          ...        ...             ...                       ...   
940     9 vs 25   window=5             NaN                     False   
941     9 vs 27  window=10             NaN                     False   
942     9 vs 27  window=15             NaN                     False   
943     9 vs 27  window=20             NaN                     False   
944     9 vs 27   window=5             NaN                     False   

     Mean P-Value 2  Proportion H0 Rejected 2  
0               NaN                     False  
1               NaN                    